In [53]:
import pandas as pd
import numpy as np
import pickle
import pandas_profiling as pp
from pandas_profiling import ProfileReport
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.tree import export_graphviz
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

In [29]:
L_cap = pd.read_csv("./data/stock_spy.csv")
L_cap = L_cap.loc[L_cap['date']>'2016-01-01']
L_cap = L_cap.loc[:, ["date", "change"]]

In [30]:
wsb_favd = pd.read_csv("./data/wsb_favd_count.csv")
wsb_sentbyday = pd.read_csv("./data/wsb_sentbydaydf_count.csv")
wsb_sentbydaypos = pd.read_csv("./data/wsb_sentbydayposdf_count.csv")
wsb_sentbydayneg = pd.read_csv("./data/wsb_sentbydaynegdf_count.csv")
wsb_econ2 = pd.read_csv("./data/wsb_daydf_count2.csv")

In [35]:
wsb_mkt = L_cap.merge(wsb_sentbydayneg, how='inner', left_on="date" ,right_on="Date")
wsb_mkt = wsb_mkt.merge(wsb_sentbydaypos, how='inner' ,on="Date")
wsb_mkt =  wsb_mkt.merge(wsb_sentbyday, how='inner' ,on="Date")
wsb_mkt =  wsb_mkt.merge(wsb_favd, how='inner' , left_on="date" ,right_on="day")
wsb_mkt =  wsb_mkt.merge(wsb_econ2, how='inner' ,on="Date")
wsb_mkt['pos_neg'] = np.where (wsb_mkt['change'] > 0,1,0)
wsb_mkt['pos'] = np.where (wsb_mkt['change'] > 0.002,1,0)
wsb_mkt['neg'] = np.where (wsb_mkt['change'] < -0.002,1,0)
wsb_mkt['flat'] = np.where ((wsb_mkt['change'] <= 0.002) & (wsb_mkt['change'] >= -0.002) ,1,0)
wsb_mkt = wsb_mkt.drop(['Date', 'day'], axis=1)

wsb_mkt.head()

,date,change,Neg_Count,Pos_Count,Sent_Count,favorites,Count,pos_neg,pos,neg,flat
0,2016-01-04,0.002644,30,28,-2,224,22,1,1,0,0
1,2016-01-05,-0.000199,27,19,-8,156,13,0,0,0,1
2,2016-01-06,0.002420,62,39,-23,664,48,1,1,0,0
3,2016-01-07,-0.006553,84,79,-5,605,45,0,0,1,0
4,2016-01-08,-0.016753,76,75,-1,206,36,0,0,1,0


In [39]:
wsb_mkt['date'] = pd.to_datetime(wsb_mkt['date'])
wsb_empsit.dtypes
wsb_mkt.dtypes

date          datetime64[ns]
change               float64
Neg_Count              int64
Pos_Count              int64
Sent_Count             int64
favorites              int64
Count                  int64
pos_neg                int32
pos                    int32
neg                    int32
flat                   int32
job_change           float64
growthgap            float64
dtype: object

In [40]:
wsb_empsit = pd.read_csv("./Data/empsit.csv")
wsb_mkt['job_change']=np.nan
for x in range(len(wsb_empsit.index)-1):
   wsb_mkt.loc[((wsb_mkt['date']>= wsb_empsit.loc[x][0]) & (wsb_mkt['date']< wsb_empsit.loc[x+1][0])),['job_change']]=wsb_empsit.loc[x][1]
wsb_mkt.head()

,date,change,Neg_Count,Pos_Count,Sent_Count,favorites,Count,pos_neg,pos,neg,flat,job_change,growthgap
0,2016-01-04,0.002644,30,28,-2,224,22,1,1,0,0,237.0,NaN
1,2016-01-05,-0.000199,27,19,-8,156,13,0,0,0,1,237.0,NaN
2,2016-01-06,0.002420,62,39,-23,664,48,1,1,0,0,237.0,NaN
3,2016-01-07,-0.006553,84,79,-5,605,45,0,0,1,0,237.0,NaN
4,2016-01-08,-0.016753,76,75,-1,206,36,0,0,1,0,273.0,NaN


In [41]:
wsb_rgdp = pd.read_csv("./Data/RGDP.csv")
wsb_rgdp.head()
wsb_rgdp['Date']=pd.to_datetime(wsb_rgdp['Date'])


In [42]:
wsb_mkt['growthgap']=np.nan
for x in range(len(wsb_rgdp.index)-1):
   wsb_mkt.loc[((wsb_mkt['date']>= wsb_rgdp.loc[x][0]) & (wsb_mkt['date']< wsb_rgdp.loc[x+1][0])),['growthgap']]=wsb_rgdp.loc[x][3]


In [43]:
wsb_mkt.head()

,date,change,Neg_Count,Pos_Count,Sent_Count,favorites,Count,pos_neg,pos,neg,flat,job_change,growthgap
0,2016-01-04,0.002644,30,28,-2,224,22,1,1,0,0,237.0,-0.5
1,2016-01-05,-0.000199,27,19,-8,156,13,0,0,0,1,237.0,-0.5
2,2016-01-06,0.002420,62,39,-23,664,48,1,1,0,0,237.0,-0.5
3,2016-01-07,-0.006553,84,79,-5,605,45,0,0,1,0,237.0,-0.5
4,2016-01-08,-0.016753,76,75,-1,206,36,0,0,1,0,273.0,-0.5


In [44]:
wsb_mkt['negcount1day'] = wsb_mkt['Neg_Count'].shift(1)
wsb_mkt['poscount1day'] = wsb_mkt['Pos_Count'].shift(1)
wsb_mkt['sentcount1day'] = wsb_mkt['Sent_Count'].shift(1)
wsb_mkt['favcount1day'] = wsb_mkt['favorites'].shift(1)
wsb_mkt['econcount21day'] = wsb_mkt['Count'].shift(1)
wsb_mkt['job_change1day'] = wsb_mkt['job_change'].shift(1)
wsb_mkt['growthgap1day'] = wsb_mkt['growthgap'].shift(1)
wsb_mkt['change1day'] = wsb_mkt['change'].shift(1)
wsb_mkt['negcount2day'] = wsb_mkt['Neg_Count'].shift(2)
wsb_mkt['poscount2day'] = wsb_mkt['Pos_Count'].shift(2)
wsb_mkt['sentcount2day'] = wsb_mkt['Sent_Count'].shift(2)
wsb_mkt['favcount2day'] = wsb_mkt['favorites'].shift(2)
wsb_mkt['econcount22day'] = wsb_mkt['Count'].shift(2)
wsb_mkt['job_change2day'] = wsb_mkt['job_change'].shift(2)
wsb_mkt['growthgap2day'] = wsb_mkt['growthgap'].shift(2)
wsb_mkt['change2day'] = wsb_mkt['change'].shift(2)
wsb_mkt['negcount3day'] = wsb_mkt['Neg_Count'].shift(3)
wsb_mkt['poscount3day'] = wsb_mkt['Pos_Count'].shift(3)
wsb_mkt['sentcount3day'] = wsb_mkt['Sent_Count'].shift(3)
wsb_mkt['favcount3day'] = wsb_mkt['favorites'].shift(3)
wsb_mkt['econcount23day'] = wsb_mkt['Count'].shift(3)
wsb_mkt['job_change3day'] = wsb_mkt['job_change'].shift(3)
wsb_mkt['growthgap3day'] = wsb_mkt['growthgap'].shift(3)
wsb_mkt['change3day'] = wsb_mkt['change'].shift(3)
wsb_mkt['negcount4day'] = wsb_mkt['Neg_Count'].shift(4)
wsb_mkt['poscount4day'] = wsb_mkt['Pos_Count'].shift(4)
wsb_mkt['sentcount4day'] = wsb_mkt['Sent_Count'].shift(4)
wsb_mkt['favcount4day'] = wsb_mkt['favorites'].shift(4)
wsb_mkt['econcount24day'] = wsb_mkt['Count'].shift(4)
wsb_mkt['job_change4day'] = wsb_mkt['job_change'].shift(4)
wsb_mkt['growthgap4day'] = wsb_mkt['growthgap'].shift(4)
wsb_mkt['change4day'] = wsb_mkt['change'].shift(4)
wsb_mkt['negcount5day'] = wsb_mkt['Neg_Count'].shift(5)
wsb_mkt['poscount5day'] = wsb_mkt['Pos_Count'].shift(5)
wsb_mkt['sentcount5day'] = wsb_mkt['Sent_Count'].shift(5)
wsb_mkt['favcount5day'] = wsb_mkt['favorites'].shift(5)
wsb_mkt['econcount25day'] = wsb_mkt['Count'].shift(5)
wsb_mkt['job_change5day'] = wsb_mkt['job_change'].shift(5)
wsb_mkt['growthgap5day'] = wsb_mkt['growthgap'].shift(5)
wsb_mkt['change5day'] = wsb_mkt['change'].shift(5)
wsb_mkt['negcount6day'] = wsb_mkt['Neg_Count'].shift(6)
wsb_mkt['poscount6day'] = wsb_mkt['Pos_Count'].shift(6)
wsb_mkt['sentcount6day'] = wsb_mkt['Sent_Count'].shift(6)
wsb_mkt['favcount6day'] = wsb_mkt['favorites'].shift(6)
wsb_mkt['econcount26day'] = wsb_mkt['Count'].shift(6)
wsb_mkt['job_change6day'] = wsb_mkt['job_change'].shift(6)
wsb_mkt['growthgap6day'] = wsb_mkt['growthgap'].shift(6)
wsb_mkt['change6day'] = wsb_mkt['change'].shift(6)
wsb_mkt['negcount7day'] = wsb_mkt['Neg_Count'].shift(7)
wsb_mkt['poscount7day'] = wsb_mkt['Pos_Count'].shift(7)
wsb_mkt['sentcount7day'] = wsb_mkt['Sent_Count'].shift(7)
wsb_mkt['favcount7day'] = wsb_mkt['favorites'].shift(7)
wsb_mkt['econcount27day'] = wsb_mkt['Count'].shift(7)
wsb_mkt['job_change7day'] = wsb_mkt['job_change'].shift(7)
wsb_mkt['growthgap7day'] = wsb_mkt['growthgap'].shift(7)
wsb_mkt['change7day'] = wsb_mkt['change'].shift(7)
wsb_mkt['negcount8day'] = wsb_mkt['Neg_Count'].shift(8)
wsb_mkt['poscount8day'] = wsb_mkt['Pos_Count'].shift(8)
wsb_mkt['sentcount8day'] = wsb_mkt['Sent_Count'].shift(8)
wsb_mkt['favcount8day'] = wsb_mkt['favorites'].shift(8)
wsb_mkt['econcount28day'] = wsb_mkt['Count'].shift(8)
wsb_mkt['job_change8day'] = wsb_mkt['job_change'].shift(8)
wsb_mkt['growthgap8day'] = wsb_mkt['growthgap'].shift(8)
wsb_mkt['change8day'] = wsb_mkt['change'].shift(8)
wsb_mkt['negcount9day'] = wsb_mkt['Neg_Count'].shift(9)
wsb_mkt['poscount9day'] = wsb_mkt['Pos_Count'].shift(9)
wsb_mkt['sentcount9day'] = wsb_mkt['Sent_Count'].shift(9)
wsb_mkt['favcount9day'] = wsb_mkt['favorites'].shift(9)
wsb_mkt['econcount29day'] = wsb_mkt['Count'].shift(9)
wsb_mkt['job_change9day'] = wsb_mkt['job_change'].shift(9)
wsb_mkt['growthgap9day'] = wsb_mkt['growthgap'].shift(9)
wsb_mkt['change9day'] = wsb_mkt['change'].shift(9)
wsb_mkt['negcount10day'] = wsb_mkt['Neg_Count'].shift(10)
wsb_mkt['poscount10day'] = wsb_mkt['Pos_Count'].shift(10)
wsb_mkt['sentcount10day'] = wsb_mkt['Sent_Count'].shift(10)
wsb_mkt['favcount10day'] = wsb_mkt['favorites'].shift(10)
wsb_mkt['econcount210day'] = wsb_mkt['Count'].shift(10)
wsb_mkt['job_change10day'] = wsb_mkt['job_change'].shift(10)
wsb_mkt['growthgap10day'] = wsb_mkt['growthgap'].shift(10)
wsb_mkt['change10day'] = wsb_mkt['change'].shift(10)
wsb_mkt['negcount11day'] = wsb_mkt['Neg_Count'].shift(11)
wsb_mkt['poscount11day'] = wsb_mkt['Pos_Count'].shift(11)
wsb_mkt['sentcount11day'] = wsb_mkt['Sent_Count'].shift(11)
wsb_mkt['favcount11day'] = wsb_mkt['favorites'].shift(11)
wsb_mkt['econcount211day'] = wsb_mkt['Count'].shift(11)
wsb_mkt['job_change11day'] = wsb_mkt['job_change'].shift(11)
wsb_mkt['growthgap11day'] = wsb_mkt['growthgap'].shift(11)
wsb_mkt['change11day'] = wsb_mkt['change'].shift(11)
wsb_mkt['negcount12day'] = wsb_mkt['Neg_Count'].shift(12)
wsb_mkt['poscount12day'] = wsb_mkt['Pos_Count'].shift(12)
wsb_mkt['sentcount12day'] = wsb_mkt['Sent_Count'].shift(12)
wsb_mkt['favcount12day'] = wsb_mkt['favorites'].shift(12)
wsb_mkt['econcount212day'] = wsb_mkt['Count'].shift(12)
wsb_mkt['job_change12day'] = wsb_mkt['job_change'].shift(12)
wsb_mkt['growthgap12day'] = wsb_mkt['growthgap'].shift(12)
wsb_mkt['change12day'] = wsb_mkt['change'].shift(12)
wsb_mkt['negcount13day'] = wsb_mkt['Neg_Count'].shift(13)
wsb_mkt['poscount13day'] = wsb_mkt['Pos_Count'].shift(13)
wsb_mkt['sentcount13day'] = wsb_mkt['Sent_Count'].shift(13)
wsb_mkt['favcount13day'] = wsb_mkt['favorites'].shift(13)
wsb_mkt['econcount213day'] = wsb_mkt['Count'].shift(13)
wsb_mkt['job_change13day'] = wsb_mkt['job_change'].shift(13)
wsb_mkt['growthgap13day'] = wsb_mkt['growthgap'].shift(13)
wsb_mkt['change13day'] = wsb_mkt['change'].shift(13)
wsb_mkt['negcount14day'] = wsb_mkt['Neg_Count'].shift(14)
wsb_mkt['poscount14day'] = wsb_mkt['Pos_Count'].shift(14)
wsb_mkt['sentcount14day'] = wsb_mkt['Sent_Count'].shift(14)
wsb_mkt['favcount14day'] = wsb_mkt['favorites'].shift(14)
wsb_mkt['econcount214day'] = wsb_mkt['Count'].shift(14)
wsb_mkt['job_change14day'] = wsb_mkt['job_change'].shift(14)
wsb_mkt['growthgap14day'] = wsb_mkt['growthgap'].shift(14)
wsb_mkt['change14day'] = wsb_mkt['change'].shift(14)
wsb_mkt['negcount15day'] = wsb_mkt['Neg_Count'].shift(15)
wsb_mkt['poscount15day'] = wsb_mkt['Pos_Count'].shift(15)
wsb_mkt['sentcount15day'] = wsb_mkt['Sent_Count'].shift(15)
wsb_mkt['favcount15day'] = wsb_mkt['favorites'].shift(15)
wsb_mkt['econcount215day'] = wsb_mkt['Count'].shift(15)
wsb_mkt['job_change15day'] = wsb_mkt['job_change'].shift(15)
wsb_mkt['growthgap15day'] = wsb_mkt['growthgap'].shift(15)
wsb_mkt['change15day'] = wsb_mkt['change'].shift(15)
wsb_mkt

,date,change,Neg_Count,Pos_Count,Sent_Count,favorites,Count,pos_neg,pos,neg,...,growthgap14day,change14day,negcount15day,poscount15day,sentcount15day,favcount15day,econcount215day,job_change15day,growthgap15day,change15day
0,2016-01-04,0.002644,30,28,-2,224,22,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-05,-0.000199,27,19,-8,156,13,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-06,0.002420,62,39,-23,664,48,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-07,-0.006553,84,79,-5,605,45,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-08,-0.016753,76,75,-1,206,36,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,2021-04-26,0.000407,406,755,349,494,560,1,0,0,...,NaN,0.005547,942.0,1234.0,292.0,559.0,825.0,NaN,NaN,0.002504
1320,2021-04-27,-0.000981,617,1137,520,558,731,0,0,0,...,NaN,0.007188,1115.0,1520.0,405.0,602.0,1114.0,NaN,NaN,0.005547
1321,2021-04-28,-0.000981,600,1104,504,452,699,0,0,0,...,NaN,0.000887,454.0,694.0,240.0,467.0,499.0,NaN,NaN,0.007188
1322,2021-04-29,-0.000619,462,805,343,455,512,0,0,0,...,NaN,0.001601,495.0,711.0,216.0,373.0,489.0,NaN,NaN,0.000887


In [54]:
profile = ProfileReport(wsb_mkt)
profile

In [55]:
profile.to_file("wsb_profile.html")

In [45]:
wsb_mkt.dropna(inplace=True)
#wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 2:4, 5:7]] #1 Neg_Count, Pos_Count
#wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 4:5, 5:7]] #2 Sent_Count
#wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 2:4, 5:7, 11:13]] #3 Neg_Count, Pos_Count, job_change, growthgap
#wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 2:4, 5:7, 11:13, 13:15, 16:23, 24:31, 32:39, 40:47, 48 -55, 56:63, 64:71, 72:79, 80:87, 88:95, 96:103, 104:111, 112:119, 120:127, 128:133]] #4 with lags
#wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 4:5, 5:6, 12:13, ]] #5 Sent_Count, growthgap
wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 4:5, 5:6, 12:13, ]] #6 Sent_Count, growthgap with lags
wsb_lmkt_x.set_index("date", inplace=True)
wsb_lmkt_y1 = wsb_mkt['pos_neg']
wsb_lmkt_y2 = wsb_mkt[['pos','neg','flat']]
wsb_lmkt_y3 = wsb_mkt[['change']]
wsb_lmkt_x.head()
#wsb_lmkt_x.to_csv('./Data/wsb_lmkt_x.csv', index=True)
#wsb_lmkt_y1.to_csv('./Data/wsb_lmkt_y1.csv', index=False)

,Sent_Count,favorites,growthgap
date,,,
2016-01-26,0,500,-0.5
2016-01-27,6,337,-0.5
2016-01-28,9,608,-0.5
2016-01-29,-3,157,-0.1
2016-02-01,-1,276,-0.1


In [46]:
X_train, X_test,y_train,y_test = train_test_split(wsb_lmkt_x, wsb_lmkt_y3, test_size = 0.33,random_state=42)
regr = RandomForestRegressor()
regr.fit(X_train,y_train)
#print(regr.score(X_test, y_test))

C:\Users\areer\Anaconda3\envs\pandas\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\areer\Anaconda3\envs\pandas\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [47]:
ridge = Ridge()
ridge.fit(X_train,y_train)
#print(ridge.score(X_test, y_test))
# regr.predict_proba(X_test)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
print(regr.score(X_test, y_test)) #6 Sent_Count, growthgap, with lags

In [ ]:
sorted(zip(regr.feature_importances_, wsb_lmkt_x), reverse=True)

In [ ]:
print(ridge.score(X_test, y_test)) #6 Sent_Count, growthgap, with lags

In [48]:
print(regr.score(X_test, y_test)) #5 Sent_Count, growthgap

-0.23147404115983528


In [50]:
sorted(zip(regr.feature_importances_, wsb_lmkt_x), reverse=True)

[(0.4172855296479018, 'favorites'),
 (0.41593833175340417, 'Sent_Count'),
 (0.16677613859869414, 'growthgap')]

In [51]:
print(ridge.score(X_test, y_test)) #5 Sent_Count, growthgap

-0.012202471341165921


In [ ]:
wsb_mkt.dropna(inplace=True)
#wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 2:4, 5:7]] #1 Neg_Count, Pos_Count
#wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 4:5, 5:7]] #2 Sent_Count
#wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 2:4, 5:7, 11:13]] #3 Neg_Count, Pos_Count, job_change, growthgap
#wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 2:4, 5:7, 11:13, 13:15, 16:23, 24:31, 32:39, 40:47, 48 -55, 56:63, 64:71, 72:79, 80:87, 88:95, 96:103, 104:111, 112:119, 120:127, 128:133]] #4 with lags
#wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 4:5, 5:6, 12:13]] #5 Sent_Count, growthgap
wsb_lmkt_x = wsb_mkt.iloc[:, np.r_[0, 4:5, 5:6, 12:13, ]] #6 Sent_Count, growthgap, with lags, #7 Sent_Count, growthgap with lags and Y1
wsb_lmkt_x.set_index("date", inplace=True)
wsb_lmkt_y1 = wsb_mkt['pos_neg']
wsb_lmkt_y2 = wsb_mkt[['pos','neg','flat']]
wsb_lmkt_y3 = wsb_mkt[['change']]
wsb_lmkt_x.head()
#wsb_lmkt_x.to_csv('./Data/wsb_lmkt_x.csv', index=True)
#wsb_lmkt_y1.to_csv('./Data/wsb_lmkt_y1.csv', index=False)

In [ ]:
X_train, X_test,y_train,y_test = train_test_split(wsb_lmkt_x, wsb_lmkt_y1, test_size = 0.33,random_state=42)
regr = RandomForestRegressor()
regr.fit(X_train,y_train)
#print(regr.score(X_test, y_test))

In [ ]:
ridge = Ridge()
ridge.fit(X_train,y_train)
#print(ridge.score(X_test, y_test))
# regr.predict_proba(X_test)

In [ ]:
print(regr.score(X_test, y_test)) #6 Sent_Count, growthgap, with lags

In [ ]:
sorted(zip(regr.feature_importances_, wsb_lmkt_x), reverse=True)

In [ ]:
print(ridge.score(X_test, y_test)) #6 Sent_Count, growthgap, with lags